## Supervised learning of  a simple genetic network in *E. coli*

Content here is licensed under a CC 4.0 License. The code in this notebook is released under the MIT license. 


By Manu Flores. 

In [ ]:
# uncomment the next line if you're in Google Collab 
#! pip install -r https://raw.githubusercontent.com/manuflores/grnlearn_tutorial/master/requirements.txt
#! wget https://raw.githubusercontent.com/manuflores/grnlearn_tutorial/master/notebooks/grn.py

In [ ]:
import grn as g
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import matplotlib as mpl
from scipy.stats import pearsonr

import hvplot
import hvplot.pandas
import holoviews as hv
from holoviews import dim, opts
import bokeh_catplot
import bokeh 
import bokeh.io
from bokeh.io import output_file, save, output_notebook


output_notebook()
hv.extension('bokeh')
seed = 8
np.random.seed(8)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

g.set_plotting_style()
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

Welcome back ! This is the core of the tutorial. In this notebook we will learn the patterns of a [gene regulatory network](https://en.wikipedia.org/wiki/Gene_regulatory_network), more specifically of the [genetic network of a single regulatory protein (PurR)](https://academic.oup.com/nar/article/39/15/6456/1022585) in *Escherichia coli*. In the last tutorial we extracted the connection of this simple network using data from [RegulonDB](http://regulondb.ccg.unam.mx/menu/about_regulondb/what_is_regulondb/index.jsp). In this tutorial we will continue using data from the Palsson Lab at UCSD. 

Now that we have extracted the PurR gene network (*PurR [regulon](https://en.wikipedia.org/wiki/Regulon)*), now it's time to prepare the dataset in order to learn the patterns that will enable us to predict new genes that might be inside this biological module. 

### Load in *E. coli* RNA-seq dataset. 

We will be using an RNA-seq dataset from the Palsson Lab published in this [paper](https://academic.oup.com/nar/article/47/5/2446/5304327). This dataset includes more than 50 expression conditions consisting of single gene mutants and laboratory evolution experiments. This dataset is nice because it contains genetic perturbations that represent different cell states and will ideally allow us to infer important biological information from the PurR system in *E. coli*. One last thing to notice is that the data are in [transcript per million](http://www.arrayserver.com/wiki/index.php?title=TPM) units and they were log-transformed before analysis. 

Let's go ahead and load the dataset into our workspace. Important to notice that if you're running this notebook in Google Colab, you will have to load the dataset directly from the Github url presented below. 

In [ ]:
# url = 'https://raw.githubusercontent.com/manuflores/grnlearn_tutorial/master/data/palsson_rna_seq.csv'
# df = pd.read_csv(url)
df = pd.read_csv('../data/palsson_rna_seq.csv')

In [ ]:
df.shape

We can see that the dataset contains 4K + rows and 100 + columns. In this dataset **each row represent a gene and each column an expression condition**. The only exceptions are the first two first columns correspond to the [locus tag](https://www.wikidata.org/wiki/Property:P2393) and the gene's name. In this sense, each column represents a proxy to the amount of RNA collected for each gene in a given experiment. The same way, you can think of every column as the intensity of "expression" of a given gene in multiple growth conditions. 

In [ ]:
df.head()

Now let's just divide the annotation and numerical segments of our dataset to continue processing the numerical data. 

In [ ]:
data_ = df.copy()

# Extracting the annotation of the dataset (gene information)
annot = data_.iloc[:, :2]

# Extracting the real numerical data log(TPM)
data = data_.iloc[:, 2:]

### Data preprocessing. 

Let's start our data analysis pipeline by normalizing and looking for null values.

In [ ]:
from sklearn.preprocessing import StandardScaler as scaler 

In [ ]:
ss = scaler()
norm_data = ss.fit_transform(data)

Let's check if the data has any null entries.

In [ ]:
norm_data= pd.DataFrame(norm_data, columns = data.columns)
norm_data.describe()

It looks like there are none. We can quickly verify this using the `pd.notnull` function from pandas.

In [ ]:
np.all(pd.notnull(norm_data))

All right, we're good to go ! 

### Load in PurR regulon datasets to annotate our training and test datasets

After normalizing our data, we want to make a training and test data sets. Let's load in the data from the last analysis. 

In [ ]:
# You know the drill, uncomment if in colab 
# url_purr_rdb = https://raw.githubusercontent.com/manuflores/grnlearn_tutorial/master/data/purr_regulon_db.csv
#purr_regulondb = pd.read_csv(url)
purr_regulondb = pd.read_csv('../data/purr_regulon_db.csv')

In [ ]:
# url_purr_hi = https://raw.githubusercontent.com/manuflores/grnlearn_tutorial/master/data/purr_regulon_db.csv
#purr_hi = pd.read_csv(url_purr_hi)
purr_hi = pd.read_csv('../data/purr_regulon_hitrn.csv')

In [ ]:
print('The RegulonDB has %d nodes and the hiTRN has %d nodes \
for the PurR regulon genetic network respectively.'%(purr_regulondb.shape[0], purr_hi.shape[0]))

As a reminder in this datasets, the `TG` column represents the **target genes** that are controlled by PurR. In other words, the genes that are directly regulated by the PurR regulator will be in the TG column of this dataframes. 

Let's extract the TGs as a `np.array` and get the genes that were discovered by the Palsson Lab. These extra genes discovered will serve as our test dataset. 

In [ ]:
# Get the target genes of the PurR gene network from RegulonDB
purr_rdb_tgs = np.unique(purr_regulondb.tg.values)

In [ ]:
len(purr_rdb_tgs)

In [ ]:
# Get the target genes of the PurR gene network from the Palsson dataset
purr_hi_tgs = np.unique(purr_hi.gene.values)

purr_hi_tgs = [gene.lower() for gene in purr_hi_tgs]

In [ ]:
# Extract the discovered genes by the Palsson lab 
new_purr_tgs = set(purr_hi_tgs) - set(purr_rdb_tgs)

new_purr_tgs

We can see that indeed the hiTRN has 5 more interactions. Let's see if we can accurately predict this interactions directly from the RNA-seq data. 

### Visualizing correlation between genes across conditions.

Before jumping to applying an ML model to our data, let's proceed to make a simple EDA. As I've said in the presentation the notion that makes this approach biologically plausible is that **genes that are coexpressed are probably corregulated**. A simple proxy for coexpression is correlation across expression conditions. **However, we're not implying that correlation indicates a regulatory interaction.** This is just to get a feel of the data.

Let's make a couple of plots to see that indeed the test genes that we're looking for are correlated with purr, and if this relationship looks linear. We'll use the Seaborn library in this case because it has a nice feat that allows to embed a statistical function into the plot. 

In [ ]:
def corr_plot(data, gene_1, gene_2):
    """
    Scatter plot to devise correlation. 
    
    Parameters
    -----------
    * data(pd.DataFrame): Input dataframe that contains for which to pull out data. 
    
    * gene_x (str): gene_name of the genes to visualize.
    
    Returns 
    ---------
    * fig (plt.figure) : sns.jointplot hardcoded to be a scatterplot of the genes. 
    
    """
    gene_1_data  = data[data['gene_name'] == gene_1]
    
    assert gene_1_data.shape[0] ==1, 'Gene 1 not in dataset'
    
    gene_1_vals =  gene_1_data.iloc[:, 3:].values.T
    
    gene_2_data  = data[data['gene_name'] == gene_2]
    
    assert gene_2_data.shape[0] ==1, 'Gene 2 not in dataset'
    
    gene_2_vals =  gene_2_data.iloc[:, 3:].values.T
    
    df_plot = pd.DataFrame({gene_1: gene_1_vals.flatten(),
                            gene_2 : gene_2_vals.flatten()})
    
    plt.figure(figsize = (6, 4))
    fig = sns.jointplot(data = df_plot, 
                  x = gene_1,
                  y = gene_2,
                  stat_func = pearsonr,
                  alpha = 0.5,
                  color = 'dodgerblue');
    
    return fig

We can now iterate over the putative TGs and plot them against PurR. In the following plots, each dot represents the expression level (in normalized log(TPM) units), a proxy for the number of mRNA counts for a given gene) of both genes in a specific expression condition. 

In [ ]:
for new_tg in new_purr_tgs: 
    
    corr_plot(df, 'purr', new_tg);

We can see that some, but not all the genes are strongly correlated with PurR. This is normal because the TRN has a lot of feedback so it could be that despite that PurR regulates a given gene, there are potentially other TFs controlling those target genes. 

### Filter noise using PCA. 

Principal component analysis is a widely used technique in unsupervised learning to perform dimensionality reduction (if you want to know more about it I highly recommend this [blog post](https://sebastianraschka.com/Articles/2015_pca_in_3_steps.html) by Sebas Raschka). One can also use PCA as a "noise reduction" technique because projecting into a (smaller) latent space and reconstructing the dataset from this space with smaller dimensionality forces the algorithm to learn important features of the data. Specifically the latent space (the principal components) will maximize the variance across the dataset. 

First, let's explore the dimensionality of our RNA-seq dataset using PCA. 

In [ ]:
from sklearn.decomposition import PCA 

In [ ]:
pca = PCA()
pca = pca.fit(norm_data)

In [ ]:
cum_exp_var = np.cumsum(pca.explained_variance_ratio_)

# look at it
plt.figure(figsize = (6,4))
plt.plot(cum_exp_var*100, color = 'dodgerblue') #because LA
plt.xlabel('Number of dimensions', fontsize= 16)
plt.ylabel('Cumulative variance percentage', fontsize = 16)
plt.title('PCA Explained Variance');

In [ ]:
print('The first five principal components explain %.2f of the variance in the dataset.'%cum_exp_var[4])

We can see that the dataset is of very small dimensionality. We can now project into this subspace that contains 95% of the variance and reconstruct the denoised dataset.

In [ ]:
pca = PCA(0.95).fit(norm_data)
latent = pca.transform(norm_data)

In [ ]:
reconstructed = pca.inverse_transform(latent)

In [ ]:
recon_df= pd.DataFrame(reconstructed, columns = data.columns)

In [ ]:
df.iloc[:, :2].shape, recon_df.shape

In [ ]:
recon_df_ = pd.concat([df.iloc[:, :2], recon_df], axis = 1)

In [ ]:
recon_df_.head()

### Visualize correlation again. 

Let's visualize the correlation of the target genes we want to discover using the denoised dataset. 

In [ ]:
for new_tg in new_purr_tgs: 
    
    corr_plot(recon_df_, 'purr', new_tg);

We can see that in the reconstructed space, we've constrained the data to have a bigger covariance. 

### Visualize in PCA space

Given that we already have the projection of our dataset into a smaller dimension, we can also visualize all of the genes in the first two principal components. 

In [ ]:
hv.Points((latent[: , 0], latent[: , 1])).opts(xlabel = 'principal component 1',
                                               ylabel = 'principal component 2',
                                               color = '#1E90FF', 
                                               size = 5, 
                                               alpha = 0.15, 
                                               padding = 0.1, 
                                               width = 400)

We cannot really see a specific structure in the first two components. Maybe a non-linear dimensionality reduction technique such as UMAP could do a better job to get the clusters in higher dimensions. We'll come back to that in the next tutorial. 

### Annotate datasets

Now that we have preprocessed our data we can proceed to annotate it. Specifically we want to label our data for each gene, if its inside the PurR regulon or not. 

First-off, let's generate our test set. We'll use a helper function that let's us filter from the dataframe. We also have the function in the `grn` module in this folder if you want to use it later. 

In [ ]:
def get_gene_data(data, gene_name_column, test_gene_list):
    
    """
    Extract data from specific genes given a larger dataframe.
    
    Parameters
    ------------
    
    * data (pd.DataFrame): large dataframe from where to filter.
    * gene_name_column (str): column to filter from in the dataset.
    * test_gene_list (array-like) : a list of genes you want to get. 
    
    Returns
    ---------
    * gene_profiles (pd.DataFrame) : dataframe with the genes you want
    
    """
    
    gene_profiles = pd.DataFrame()

    for gene in data[gene_name_column].values:

        if gene in test_gene_list: 

            df_ = data[(data[gene_name_column] == gene)]

            gene_profiles = pd.concat([gene_profiles, df_])
    
    gene_profiles.drop_duplicates(inplace = True)
    
    return gene_profiles 

Let's make a one hot encoded vector that corresponds to being an element of the PurR regulon. 

In [ ]:
one_hot = [1 if row  in purr_hi_tgs else 0 for  row in  recon_df_['gene_name'].values]

Now let's add the one hot vector to the dataset. 

In [ ]:
# Appending the one hot vector to the dataset
recon_df_['output'] = one_hot

In [ ]:
recon_df_.head()

Now we can go ahead and make the test set using the `get_gene_data` function and the TG list to discover. 

In [ ]:
test_purr_tgs  = list(new_purr_tgs)

In [ ]:
test = get_gene_data(recon_df_, 'gene_name', test_purr_tgs)

In [ ]:
test.head()

Let's drop these test genes from the reconstructed dataset. 

In [ ]:
recon_df_non_regulon = recon_df_.copy().drop(test.index.to_list())

Nice! Finally, let's go ahead and add some "noise" to our test dataset, in the sense that we need to test if our algorithm can point out negative examples. 

In [ ]:
noise = recon_df_non_regulon.sample(n = 30, replace = False,
                         axis = 0, random_state = 42)

Let's merge both of this dataframes to get an "unbiased test set". 

In [ ]:
df_test_unb = pd.concat([test, noise]) ## unbiased test 

In [ ]:
df_test_unb.shape

In [ ]:
df_test_unbiased = df_test_unb.copy().reset_index(drop= True)

In [ ]:
df_test_unbiased.head()

In [ ]:
df_test_unbiased.shape

In [ ]:
df_train = recon_df_non_regulon.copy()

### Train - test split


All right, we're almost there. Literally all we have to do now is just divide into the training data and the target that will be the output of our supervised learning model.

In [ ]:
df_train.head()
df_test_unbiased.head()

In [ ]:
df_train.shape
df_test_unbiased.shape

In [ ]:
# Divide our training set X -> input and y-> output datasets
X_train = df_train.iloc[:, 2: -1].values
y_train = df_train.iloc[:,  -1].values

In [ ]:
# Check we did it right
X_train[:5, :5]
y_train[:5]

In [ ]:
# Divide our test set too
X_test = df_test_unbiased.iloc[:, 2:-1].values

y_test = df_test_unbiased.iloc[:, -1].values

In [ ]:
X_test[:5, :5]
y_test[:5]

Awesome, we're ready to try out different models ! 

### Balance dataset using SMOTE

Or are we? One last thing that we might want to check (and correct for) is if we have a so-called balanced training set, i.e. if we have the same number of positive (inside genetic network) and negative (not in gene network) examples. 

In [ ]:
pd.Series(y_train).value_counts()

In [ ]:
pd.Series(y_test).value_counts()

Of course we don't - because the PurR gene network only contains a tiny fraction of the whole *E. coli* genome. Luckily there are great libraries out there that can help us to balance our dataset. One of such libraries is [`imbalanced-learn`](https://imbalanced-learn.readthedocs.io/). I highly recommend this library! It is super well documented and has some really cool algorithms to over/undersample. Because we have just a tiny bit of data for the positive examples, we'll go with oversampling. A classic algorithm to do this is called SMOTE and it is based on generating new datapoints using a kNN like procedure of the positive samples. Let's transform our dataset! 


In [ ]:
from imblearn.over_sampling import SMOTE

#resampling is done on training dataset only
X_train_res, y_train_res = SMOTE(random_state = seed).fit_sample(X_train, y_train)

Now we can check that our dataset is indeed balanced.

In [ ]:
pd.Series(y_train_res).value_counts()

Awesome! This time for sure, let's apply some ML to check if we can really learn new nodes for our gene network. 

### Using supervised learning models to learn the PurR regulon. 

All right, a good thing at this point would be to apply a bunch of models, see which one performs the best within certain criterion (model complexity, runtime, etc.). Afterwards we would do some type of hyperparameter tuning and cross-validation to make our final model. 

The approach we're going to take though is the following: because we know *a priori* this is a simple genetic network and, we have a decent amount of data, we'll try a linear model first, specifically a linear Support Vector Machine. Afterwards we'll try some non-linear models like Random Forest classifier and a neural network. 

We will use the scikit learn and keras libraries for this. 

### Trying out a linear classifier. 

Let's import our linear SVM and check it's performance. 

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
linear_svm_clf = LinearSVC(random_state = seed)

In [ ]:
linear_svm_clf.fit(X_train_res, y_train_res)

In [ ]:
predictions = linear_svm_clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test, predictions)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, predictions))

In [ ]:
predictions == y_test

We can see that the linear model does quite well as we expected. At this point I would just tune the hyperparams of the model and stick to it. However, let's just try other models to have a comparison. 

### Random forest

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
ada = AdaBoostClassifier(random_state = seed)

In [ ]:
ada.fit(X_train, y_train)

In [ ]:
ada_pred = ada.predict(X_test)

In [ ]:
print(classification_report(y_test, ada_pred))

The random forest performs well too, but perhaps this is too complex of a model and it might actually be overfitting the data. In this sense I would trust the LinearSVM more and discard this one.

### Keras neural net. 

Finally, let's try out a neural network model. 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.metrics import categorical_accuracy

In [ ]:
X_test.shape[1]

Now we can implement the keras model. 

In [ ]:
model = Sequential()
model.add(Dense(units=64, activation='softmax', input_dim= X_test.shape[1]))
model.add(Dense(units=1)) # one output
model.compile(loss='mse', optimizer='RMSprop', metrics= ['accuracy'])

history = model.fit(X_train_res, y_train_res, epochs=10, batch_size=32)
accuracy = history.history['accuracy']

You've probably got an error if you were using binder, don't worry too much about it. 

In [ ]:
accuracy[-1]

In this case a simple neural network with one hidden layer and 64 neurons does pretty well and doesn't overfit our data. We could alternatively go with this model, but just for the sake of this tutorial, let's continue sticking with our LinearSVM. In practice you could continue with either one. 

### Cross-validation

Last but not least, let's perform cross-validation on our linear model to be confident about it. 

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cross_val_score(linear_svm_clf,
                X_train, y_train, 
                cv = 5)

We can see that it performs pretty good in the cross validation. 

### Make pipeline

Finally, to take this all the way to the finish line, let's do a simple pipeline that normalizes and applies the LinearSVM. With this we will tell that doing the noise reduction is not essential for our classification purposes.

In [ ]:
from sklearn.pipeline import make_pipeline

In [ ]:
df_train.head()
df_test_unbiased.head()

In [ ]:
df_master = pd.concat([df_train, df_test_unbiased])

In [ ]:
df_master.tail()

In [ ]:
pipe = make_pipeline(scaler(), LinearSVC())

In [ ]:
pipe

In [ ]:
pipe.fit(X_train, y_train)

In [ ]:
preds = pipe.predict(X_test)

In [ ]:
preds == y_test

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
sns.heatmap(confusion_matrix(y_test, preds) / confusion_matrix(y_test, preds).sum(axis = 0),
                             cmap = 'viridis_r', cbar_kws = {'label': 'fraction of predictions'})

plt.xlabel('predicted label')
plt.ylabel('predicted label');